In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
from random import randint
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
from PIL import Image
import seaborn as sns

def plotImages( images_arr, n_images=4):
    fig, axes = plt.subplots(n_images, n_images, figsize=(12,12))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        if img.ndim != 2:
            img = img.reshape( (SIZE,SIZE))
        ax.imshow( img, cmap="Greys_r")
        ax.set_xticks(())
        ax.set_yticks(())
    plt.tight_layout()



datagen = ImageDataGenerator(validation_split=0.2)#rescale=1./255,


# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


#Exploring the data 
train_path = "../input/state-farm-distracted-driver-detection/imgs/train/"
test_path = "../input/state-farm-distracted-driver-detection/imgs/test/"

# VISUALIZATION
category_names = os.listdir(train_path)
nb_categories = len(category_names) 
train_images = []

fig, axs = plt.subplots(ncols=1)

for category in category_names:
    folder = train_path + "/" + category
    train_images.append(len(os.listdir(folder)))

sns.barplot( y=category_names, x=train_images).set_title("Number Of Training Images Per Category");


# Load training and validation sets
train_generator = datagen.flow_from_directory(
    train_path,
    class_mode='categorical',
    target_size=(150, 150),
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    train_path,
    class_mode='categorical',
    target_size=(150, 150),
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    test_path,
    class_mode='categorical',
    target_size=(150, 150),
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)


#model intialise 
model = keras.Sequential([
    layers.InputLayer(input_shape=[150,150, 3]),
    
    #preprocessing.RandomContrast(factor=0.5),
    #preprocessing.RandomFlip(mode='horizontal'), # meaning, left-to-right
    #preprocessing.RandomContrast(factor=0.5),
    #preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),

    #preprocessing.RandomWidth(factor=0.15), # horizontal stretch

    # Block One
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.Dropout(0.3),
    layers.MaxPool2D(),

    # Block Two
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.Dropout(0.3),
    layers.MaxPool2D(),

    # Block Three
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Dropout(0.3),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    
    # Block Four
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Dropout(0.3),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),
    # Head
    layers.BatchNormalization(renorm=True),
    layers.Flatten(),
    layers.Dense(10, activation='softmax'),
])

#compile model
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=3,
)


# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['categorical_accuracy', 'val_categorical_accuracy']].plot();
